In [1]:
import numpy as np
import scipy as sp
import igl
import trimesh
import polyscope as ps
import os

root_folder = os.getcwd()

### TODO? Make the demos interactive (e.g. select source vertices for heat geodisics)

In [2]:
[V,F]= igl.read_triangle_mesh(os.path.join(
            root_folder, "../data", "spot.obj"))

In [3]:
#init poly scope and show mesh
ps.init()
ps.set_up_dir('z_up')
ps.set_ground_plane_mode('tile_reflection')

#register mesh
#mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0], edge_width=0.9, edge_color=[0.2, 0.1, 0.8, 1.0])
#ps.show()

# Solve Poisson equation (interpolation)

In [5]:
# use cotan Laplace and Mass Matrix to solve Lx = -M(rho - rho0)

L = igl.cotmatrix(V, F)
M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)

# choose 2 random vertex indices
v1 = np.random.randint(0, V.shape[0])
v2 = np.random.randint(0, V.shape[0])

# rho is a column vector of length V.shape[0] and with all zeros except for rho[v1] and rho[v2]
rho = sp.zeros((1, V.shape[0]))
rho[:,v1] = 100
rho[:,v2] = -100

def solvePoisson(L, M, rho):
        totArea = sp.sum(M.diagonal())

        # rho0 is a column vector of length V.shape[0], with all the same value = sum(rho[i] * M[i,i]) / totArea
        rho0 = sp.sum(rho * M.diagonal()) / totArea # this is more general than just summing at the 2 vertices

        f = sp.sparse.linalg.spsolve(L, -M.dot(rho.T - rho0.T))

        return f

f = solvePoisson(L, M, rho)

# plot the result
mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0])
mesh.add_scalar_quantity("f", 
        f, defined_on='vertices', cmap="coolwarm")

ps.show()




C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_1400/3034217842.py:11: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead
  rho = sp.zeros((1, V.shape[0]))
C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_1400/3034217842.py:16: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  totArea = sp.sum(M.diagonal())
C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_1400/3034217842.py:19: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  rho0 = sp.sum(rho * M.diagonal()) / totArea # this is more general than just summing at the 2 vertices


## Mesh Smoothing

In [5]:
# use cotan Laplace matrix for smoothing
# solving (M - h * L) Vnew = M * Vold

L = igl.cotmatrix(V, F)
h = igl.avg_edge_length(V, F) * 0.1

t = 0
n = 15

def smooth():
    global t
    global n
    global V
    global mesh

    if t < n:
        M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
        A = M - h * L

        for i in range(3):
            V[:,i] = sp.sparse.linalg.spsolve(A, M.dot(V[:,i])) # Todo look for a way to use a positive semi-definite solver

        mesh.update_vertex_positions(V)
        t += 1
        print("Smoothing iteration: ", t)

ps.set_user_callback(smooth)

ps.show()

ps.clear_user_callback()



Smoothing iteration:  1
Smoothing iteration:  2
Smoothing iteration:  3
Smoothing iteration:  4
Smoothing iteration:  5
Smoothing iteration:  6
Smoothing iteration:  7
Smoothing iteration:  8
Smoothing iteration:  9
Smoothing iteration:  10
Smoothing iteration:  11
Smoothing iteration:  12
Smoothing iteration:  13
Smoothing iteration:  14
Smoothing iteration:  15


## Next: Heat Method for Geodesic Distances

In [27]:
# L = igl.cotmatrix(V, F)
# M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
# h = 0.005
# A = M + h * L

# print(sp.sparse.linalg.norm(M))
# I = sp.sparse.identity(M.shape[0])
# print(I.dot(V[:, 0]) - V[:, 0])
# print(sp.sparse.linalg.norm(sp.sparse.identity(M.shape[0])))


0.08031036303393863
[0. 0. 0. ... 0. 0. 0.]
119.54078801814885
